In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,infinitely,new,west,myself,ives,whistle,solitary,mention,village,patiently,...,carefully,root,bag,news,mamma,cup,frightful,fight,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.40,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

NameError: name 'word_counts' is not defined

Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.957993730408

Test set score: 0.915883458647


In [9]:
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier()
train = rc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', rc.score(X_train, y_train))
print('\nTest set score:', rc.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.963009404389

Test set score: 0.890977443609


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.884639498433

Test set score: 0.873590225564


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [8]:
# Clean the Emma data.
sense = gutenberg.raw('shakespeare-hamlet.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

Chapter 1 Sir Walter Elliot, of Kellynch Hall, in Somersetshire, was a man who, for his own amusemen


In [9]:
# Parse our cleaned data.
sense_doc = nlp(sense)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

## Setting up spaCy on Shakespear Macbeth

In [8]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [9]:
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth) 
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [10]:
# Parse our cleaned data.
macbeth_doc = nlp(macbeth)

## Setting up spaCy on Melville's Moby Dick

In [12]:
parents = gutenberg.raw('edgeworth-parents.txt')
parents = re.sub(r'VOLUME \w+', '', parents)
parents = re.sub(r'CHAPTER \w+', '', leaves)
parents = text_cleaner(leaves) 
print(leaves[:100])

ETYMOLOGY. (Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher threadbare in c


In [13]:
# Parse our cleaned data.
moby_doc = nlp(moby)

ValueError: [E088] Text of length 1207515 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

Process novels 

alice_doc
persuasion_doc
macbeth_doc
moby_doc

In [22]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)

In [26]:
word_frequencies(macbeth_doc, include_stop=False).most_common(10)

[('I', 330),
 ('And', 170),
 ('Macb', 137),
 ("'s", 131),
 ('The', 117),
 ('haue', 117),
 ('Enter', 80),
 ('That', 80),
 ('What', 74),
 ('To', 73)]

In [30]:
entities = list(macbeth_doc.ents)
for entity in entities:
    print(entity.label_, ' '.join(t.orth_ for t in entity))

NORP Lightning
CARDINAL three
PERSON Witches
CARDINAL 1
CARDINAL three
ORG Thunder , Lightning
PERSON Raine
CARDINAL 2
PERSON Hurley
ORG Battaile
CARDINAL 3
DATE Sunne 1
CARDINAL 2
CARDINAL 3
CARDINAL 1
PERSON Gray - Malkin
PERSON Scena Secunda
ORG Alarum
ORG King Malcome
ORG Donalbaine
PERSON Lenox
ORG Captaine
ORDINAL seemeth
PERSON Reuolt
PERSON Mal
NORP Serieant
PERSON Souldier
GPE Captiuitie
PERSON Haile
ORG Say
GPE Broyle
PERSON Cap
CARDINAL two
PERSON Swimmers
PERSON Macdonwald
GPE Worthie
PERSON Rebell
PERSON the Westerne Isles Of Kernes
PERSON Gallowgrosses
WORK_OF_ART Fortune on
ORG Quarry
PERSON Shew'd
PERSON Rebells Whore
PERSON Macbeth
PERSON Name
PERSON Disdayning Fortune
PERSON Steele
PERSON Valours Minion
ORG Naue
PERSON Battlements King
GPE Cousin
PERSON Gentleman Cap
PERSON Sunne
PERSON Stormes
ORG Spring
PERSON Discomfort
PERSON Marke King
GPE Scotland
PERSON Iustice
PERSON Valour
ORG Compell'd
PRODUCT Kernes
PERSON Norweyan Lord
PERSON Armes
PERSON Began
GPE Captain

PERSON Banquo
PERSON Macb
PERSON Mur
PERSON Mac
GPE Thou
EVENT Cut - throats
NORP Fleans
NORP Non
GPE Mur
NORP Fleans
PERSON Macb
PRODUCT Fit
ORG Whole
FAC Marble
GPE Rocke
PERSON Ayre
PERSON Banquo
PERSON Mur
CARDINAL twenty
PERSON Serpent
PERSON Hath Nature
ORG Venom
DATE morrow
PERSON Wee'l
FAC Exit Murderer
ORG Royall
FAC Cheere
ORG Ceremony , Meeting
PERSON Banquo
GPE Macbeths
PERSON Macb
PERSON Sweet Remembrancer
ORG Appetite
PERSON Lenox
GPE Highnesse
PERSON Macb
PERSON Countries Honor
PERSON Banquo
ORG Mischance Rosse
PERSON Layes
GPE Highnesse
ORG your Royall Company
PERSON Macb
PERSON Lenox
ORG Heere
PERSON Macb
PERSON Lenox
GPE Highnesse
PERSON Macb
PERSON Macb
GPE Rosse
GPE Highnesse
PERSON Lady
PERSON Seat
PERSON Feed
PERSON Macb
PERSON Ayre
PERSON Duncan
NORP Winters
PERSON Grandam
PERSON Macb
ORG Prythee
PERSON Behold
PERSON Charnell
PERSON Graues
PERSON Monuments Shall
ORG Mawes of Kytes La.
PERSON Macb
PERSON La. Fie
PERSON Macb
PERSON Ere
PERSON Braines
CARDINAL twent

In [38]:
mac_people = [entity.text for entity in list(macbeth_doc.ents) if entity.label_ == "PERSON"]
alice_people = [entity.text for entity in list(alice_doc.ents) if entity.label_ == "PERSON"]
persuasion_people = [entity.text for entity in list(persuasion_doc.ents) if entity.label_ == "PERSON"]

In [39]:
mac_sents = [[sent, "Macbeth"] for sent in macbeth_doc.sents]
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

In [40]:
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [41]:
sentences[1].value_counts()

Austen     3649
Carroll    1669
Name: 1, dtype: int64

In [42]:
common_people = set(alice_people + persuasion_people)

In [43]:
def bow_features(sentences, common_people):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_people)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_people] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_people
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [44]:
people_counts = bow_features(sentences, common_people)
people_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,Mrs Wallis of Marlborough,the March Hare,Charles II,Morcar,Hayters,Sophys,Marlborough Buildings,Strafford,Emma,Hamilton,...,Miss Atkinson,Captain Benwick's,Archibald,Duck,Miss Musgrove,Trent,Soup,Shepherd,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [76]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.40,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.988401253918

Test set score: 0.881109022556


In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 258) (3190,)
Training set score: 0.682445141066

Test set score: 0.691729323308


In [14]:
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier()
train = rc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', rc.score(X_train, y_train))
print('\nTest set score:', rc.score(X_test, y_test))

(3190, 258) (3190,)
Training set score: 0.682445141066

Test set score: 0.691729323308


In [15]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.682445141066

Test set score: 0.691729323308


In [16]:
# Your code here. Experiment with hidden layers to build your own model.
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
mlp = MLPClassifier(hidden_layer_sizes=(3000), learning_rate='adaptive')
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=3000, learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [17]:
cross_val_score(mlp, X_test, y_test, cv=5)

array([ 0.69086651,  0.69248826,  0.69176471,  0.69176471,  0.69176471])

In [27]:
alice_gpe = [entity.text for entity in list(alice_doc.ents) if entity.label_ == "GPE"]

In [28]:
persuasion_gpe = [entity.text for entity in list(persuasion_doc.ents) if entity.label_ == "GPE"]

In [29]:
common_gpe = set(alice_gpe + persuasion_gpe)

In [30]:
def bow_features(sentences, common_gpe):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_gpe)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_gpe] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_gpe
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [31]:
word_counts = bow_features(sentences, common_gpe)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,Frederick,Portsmouth,Rome,Hand,Dormouse,Westgate Buildings,Durands,Coils,Dinn,Belmont,...,England,Kellynch,Dinah,Turtle,Captain,France,Geography,Gloucester,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [32]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.40,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.682445141066

Test set score: 0.691729323308


In [69]:
word_counts_m = word_counts.drop(word_counts[['text_sentence','text_source']], axis=1)

In [75]:
df = pd.concat([people_counts,word_counts_m],axis=1)